In [2]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import cophenet, dendrogram, linkage
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import fcluster
from streamlit_folium import folium_static
import folium
import plotly.figure_factory as ff
from folium import plugins
from folium.plugins import HeatMap
import plotly_express as px
import plotly.graph_objects as go
import plotly.express as px


In [3]:
# Read data
df = pd.read_csv("UPDATE-Lengkap-Dataset_Longsor 2021 - 2023 - PROV - Copy.csv")

In [4]:
# Memfilter variabel yang akan digunakan
df_filtered = df[["KABUPATEN", "TAHUN", "JUMLAH_LONGSOR", "JIWA_TERDAMPAK", "JIWA_MENINGGAL", "RUSAK_TERDAMPAK", "RUSAK_RINGAN", "RUSAK_SEDANG", "RUSAK_BERAT", "LATITUDE", "LONGITUDE"]]


In [5]:
df.head()

,No,KABUPATEN,TAHUN,JUMLAH_LONGSOR,JIWA_TERDAMPAK,JIWA_MENINGGAL,RUSAK_TERDAMPAK,RUSAK_RINGAN,RUSAK_SEDANG,RUSAK_BERAT,...,LAHAN,BANGUNAN LAINNYA,SARANA PENDIDIKAN,FASILITAS UMUM,LATITUDE,LONGITUDE,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,1,KABUPATEN BOGOR,2021,225,2673,0,405,213,143,49,...,0,9,5,37,-6.479036,106.824545,NaN,NaN,NaN,NaN
1,2,KABUPATEN SUKABUMI,2021,147,311,3,82,24,39,19,...,27,4,4,50,-6.988837,106.550721,NaN,NaN,NaN,NaN
2,3,KABUPATEN CIANJUR,2021,27,227,1,48,14,24,10,...,800,2,1,17,-6.822804,107.139363,NaN,NaN,NaN,NaN
3,4,KABUPATEN BANDUNG,2021,60,285,3,44,17,13,14,...,30,18,1,15,-7.021893,107.527727,NaN,NaN,NaN,NaN
4,5,KABUPATEN GARUT,2021,42,294,0,17,1,6,10,...,6,5,4,20,-7.202182,107.887006,NaN,NaN,NaN,NaN


In [6]:
df.describe()

,No,TAHUN,JUMLAH_LONGSOR,JIWA_TERDAMPAK,JIWA_MENINGGAL,RUSAK_TERDAMPAK,RUSAK_RINGAN,RUSAK_SEDANG,RUSAK_BERAT,TERTIMBUN,LAHAN,BANGUNAN LAINNYA,SARANA PENDIDIKAN,FASILITAS UMUM,LATITUDE,LONGITUDE,Unnamed: 17,Unnamed: 18,Unnamed: 19
count,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,0.0,0.0,0.0
mean,41.000000,2022.000000,25.135802,231.000000,1.012346,54.604938,22.950617,20.358025,11.296296,1.716049,11.358025,8.172840,1.320988,13.666667,-6.833017,107.686291,NaN,NaN,NaN
std,23.526581,0.821584,39.299094,465.550857,1.806196,98.623739,42.653517,40.406469,21.394768,7.386194,88.841053,13.078408,2.944602,20.409557,0.368131,0.630241,NaN,NaN,NaN
min,1.000000,2021.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-7.700090,106.550721,NaN,NaN,NaN
25%,21.000000,2021.000000,2.000000,5.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-7.021893,107.139363,NaN,NaN,NaN
50%,41.000000,2022.000000,8.000000,44.000000,0.000000,14.000000,4.000000,6.000000,2.000000,0.000000,0.000000,2.000000,0.000000,5.000000,-6.835020,107.609837,NaN,NaN,NaN
75%,61.000000,2023.000000,32.000000,283.000000,2.000000,63.000000,24.000000,20.000000,11.000000,0.000000,1.000000,9.000000,1.000000,20.000000,-6.557067,108.323123,NaN,NaN,NaN
max,81.000000,2023.000000,225.000000,2673.000000,7.000000,479.000000,213.000000,220.000000,132.000000,61.000000,800.000000,68.000000,18.000000,134.000000,-6.235769,108.563810,NaN,NaN,NaN


In [7]:
df.head()

,No,KABUPATEN,TAHUN,JUMLAH_LONGSOR,JIWA_TERDAMPAK,JIWA_MENINGGAL,RUSAK_TERDAMPAK,RUSAK_RINGAN,RUSAK_SEDANG,RUSAK_BERAT,...,LAHAN,BANGUNAN LAINNYA,SARANA PENDIDIKAN,FASILITAS UMUM,LATITUDE,LONGITUDE,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,1,KABUPATEN BOGOR,2021,225,2673,0,405,213,143,49,...,0,9,5,37,-6.479036,106.824545,NaN,NaN,NaN,NaN
1,2,KABUPATEN SUKABUMI,2021,147,311,3,82,24,39,19,...,27,4,4,50,-6.988837,106.550721,NaN,NaN,NaN,NaN
2,3,KABUPATEN CIANJUR,2021,27,227,1,48,14,24,10,...,800,2,1,17,-6.822804,107.139363,NaN,NaN,NaN,NaN
3,4,KABUPATEN BANDUNG,2021,60,285,3,44,17,13,14,...,30,18,1,15,-7.021893,107.527727,NaN,NaN,NaN,NaN
4,5,KABUPATEN GARUT,2021,42,294,0,17,1,6,10,...,6,5,4,20,-7.202182,107.887006,NaN,NaN,NaN,NaN


In [8]:
# Choose the column for the line chart
selected_column = 'JUMLAH_LONGSOR'

In [9]:
# Calculate quartiles
quartiles = df[selected_column].quantile([0.25, 0.5, 0.75])

In [10]:
# Create line chart for each quartile
fig_q1 = px.line(df, x=df.index, y=selected_column, title="Line Chart - 25th Percentile (Q1)")
fig_q1.update_layout(height=300, width=400)  # Adjust the size
fig_q1.show()



# Print quartile information
print(f"- 0.25% Percentile (Q1): {quartiles[0.25]}")


- 0.25% Percentile (Q1): 2.0


In [11]:
fig_q2 = px.line(df, x=df.index, y=selected_column, title="Line Chart - 50th Percentile (Q2)")
fig_q2.update_layout(height=300, width=400)  # Adjust the size
fig_q2.show()

print(f"- 50th Percentile (Q2): {quartiles[0.5]}")


- 50th Percentile (Q2): 8.0


In [12]:

fig_q3 = px.line(df, x=df.index, y=selected_column, title="Line Chart - 75th Percentile (Q3)")
fig_q3.update_layout(height=300, width=400)  # Adjust the size
fig_q3.show()

print(f"- 75th Percentile (Q3): {quartiles[0.75]}")


- 75th Percentile (Q3): 32.0


In [13]:
# Exploring variables
selected_features = [
    'JUMLAH_LONGSOR',
    'JIWA_TERDAMPAK',
    'JIWA_MENINGGAL',
    'RUSAK_TERDAMPAK',
    'RUSAK_RINGAN',
    'RUSAK_SEDANG',
    'RUSAK_BERAT',
    'TERTIMBUN',
    'LATITUDE',
    'LONGITUDE'
]



In [14]:
# Calculate correlation matrix
correlation_matrix = df[selected_features].corr()

# Plot heatmap using Plotly Express
fig = px.imshow(correlation_matrix,
                labels=dict(x="Features", y="Features", color="Correlation"),
                x=selected_features,
                y=selected_features,
                color_continuous_scale="viridis",
                title="Heatmap Korelasi")

fig.show()

print("Visualisasi ini memberikan gambaran distribusi univariat dari setiap variabel dalam dataset. Histogram menunjukkan sebaran nilai-nilai di setiap variabel, dan kernel density estimation (KDE) memberikan perkiraan kurva distribusi.")


Visualisasi ini memberikan gambaran distribusi univariat dari setiap variabel dalam dataset. Histogram menunjukkan sebaran nilai-nilai di setiap variabel, dan kernel density estimation (KDE) memberikan perkiraan kurva distribusi.


In [15]:
df.isnull().sum()


No                    0
KABUPATEN             0
TAHUN                 0
JUMLAH_LONGSOR        0
JIWA_TERDAMPAK        0
JIWA_MENINGGAL        0
RUSAK_TERDAMPAK       0
RUSAK_RINGAN          0
RUSAK_SEDANG          0
RUSAK_BERAT           0
TERTIMBUN             0
LAHAN                 0
BANGUNAN LAINNYA      0
SARANA PENDIDIKAN     0
FASILITAS UMUM        0
LATITUDE              0
LONGITUDE             0
Unnamed: 17          81
Unnamed: 18          81
Unnamed: 19          81
Unnamed: 20          80
dtype: int64

In [16]:

df.describe().T

,count,mean,std,min,25%,50%,75%,max
No,81.0,41.000000,23.526581,1.000000,21.000000,41.000000,61.000000,81.000000
TAHUN,81.0,2022.000000,0.821584,2021.000000,2021.000000,2022.000000,2023.000000,2023.000000
JUMLAH_LONGSOR,81.0,25.135802,39.299094,0.000000,2.000000,8.000000,32.000000,225.000000
JIWA_TERDAMPAK,81.0,231.000000,465.550857,0.000000,5.000000,44.000000,283.000000,2673.000000
JIWA_MENINGGAL,81.0,1.012346,1.806196,0.000000,0.000000,0.000000,2.000000,7.000000
RUSAK_TERDAMPAK,81.0,54.604938,98.623739,0.000000,2.000000,14.000000,63.000000,479.000000
RUSAK_RINGAN,81.0,22.950617,42.653517,0.000000,0.000000,4.000000,24.000000,213.000000
RUSAK_SEDANG,81.0,20.358025,40.406469,0.000000,1.000000,6.000000,20.000000,220.000000
RUSAK_BERAT,81.0,11.296296,21.394768,0.000000,0.000000,2.000000,11.000000,132.000000
TERTIMBUN,81.0,1.716049,7.386194,0.000000,0.000000,0.000000,0.000000,61.000000


In [17]:
#menyiapkan features untuk di cluster
X_ahc = df[['JUMLAH_LONGSOR', 'JIWA_TERDAMPAK', 'JIWA_MENINGGAL', 'RUSAK_TERDAMPAK', 'RUSAK_RINGAN', 'RUSAK_SEDANG', 'RUSAK_BERAT', 'TERTIMBUN']]

In [18]:
numeric_data = X_ahc[['JUMLAH_LONGSOR', 'JIWA_TERDAMPAK', 'JIWA_MENINGGAL', 'RUSAK_TERDAMPAK', 'RUSAK_RINGAN', 'RUSAK_SEDANG', 'RUSAK_BERAT', 'TERTIMBUN']]

# LINKAGE SINGLE

In [19]:
linkage_matrix_single = linkage(numeric_data, method='single')
cophenet_matrix_single, _ = cophenet(linkage_matrix_single, pdist(numeric_data))
ccc_single = cophenet_matrix_single.mean()
print("Cophenetic Correlation Coefficient (CCC) untuk metode 'single':", ccc_single)


Cophenetic Correlation Coefficient (CCC) untuk metode 'single': 0.9473590573572905


In [20]:
silhouette_scores = []
n_clusters_range = range(2, 11)
for n_clusters in n_clusters_range:
    clusterer_single = AgglomerativeClustering(n_clusters=n_clusters, linkage='single')
    cluster_labels = clusterer_single.fit_predict(numeric_data)
    silhouette_avg = silhouette_score(numeric_data, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"Silhouette Score untuk metode 'single' dengan {n_clusters} clusters: {silhouette_avg}")

# Menentukan jumlah cluster optimal
optimal_clusters = n_clusters_range[silhouette_scores.index(max(silhouette_scores))]
print(f"Jumlah cluster optimal berdasarkan Silhouette Score adalah: {optimal_clusters}")


Silhouette Score untuk metode 'single' dengan 2 clusters: 0.8831295055993396
Silhouette Score untuk metode 'single' dengan 3 clusters: 0.8133013977269692
Silhouette Score untuk metode 'single' dengan 4 clusters: 0.809411415306873
Silhouette Score untuk metode 'single' dengan 5 clusters: 0.8027400963865227
Silhouette Score untuk metode 'single' dengan 6 clusters: 0.2551395039337908
Silhouette Score untuk metode 'single' dengan 7 clusters: 0.2886772823641257
Silhouette Score untuk metode 'single' dengan 8 clusters: 0.2682532916457807
Silhouette Score untuk metode 'single' dengan 9 clusters: 0.27184426818287105
Silhouette Score untuk metode 'single' dengan 10 clusters: 0.22113402087622275
Jumlah cluster optimal berdasarkan Silhouette Score adalah: 2


In [21]:
clusterer_single_optimal = AgglomerativeClustering(n_clusters=optimal_clusters, linkage='single')
X_ahc_single = X_ahc.copy()
X_ahc_single['cluster_single'] = clusterer_single_optimal.fit_predict(numeric_data)


In [22]:
# Linkage Matrix
linkage_df_single = pd.DataFrame(linkage_matrix_single, columns=['Cluster1', 'Cluster2', 'Distance', 'SampleCount'])
linkage_df_single['Cluster1'] = linkage_df_single['Cluster1'].astype(int)
linkage_df_single['Cluster2'] = linkage_df_single['Cluster2'].astype(int)
linkage_csv_path = 'linkage_matrix_single.csv'
linkage_df_single.to_csv(linkage_csv_path, index=False)
print(f"Linkage Matrix (Single Linkage) telah disimpan ke: {linkage_csv_path}")

# Centroid
centroids_single = X_ahc_single.groupby('cluster_single')[['JUMLAH_LONGSOR']].mean().reset_index()
centroids_csv_path = 'centroids_single.csv'
centroids_single.to_csv(centroids_csv_path, index=True)
print(f"Centroid untuk setiap cluster telah disimpan ke: {centroids_csv_path}")


Linkage Matrix (Single Linkage) telah disimpan ke: linkage_matrix_single.csv
Centroid untuk setiap cluster telah disimpan ke: centroids_single.csv


In [23]:
centroids_single

,cluster_single,JUMLAH_LONGSOR
0,0,133.250000
1,1,19.519481


In [24]:
linkage_df_single

,Cluster1,Cluster2,Distance,SampleCount
0,22,23,0.000000,2.0
1,11,47,0.000000,2.0
2,53,65,0.000000,2.0
3,81,82,1.000000,4.0
4,26,84,1.000000,5.0
...,...,...,...,...
75,70,155,174.851365,77.0
76,28,54,342.204617,2.0
77,0,157,458.282664,3.0
78,27,158,480.009375,4.0


In [25]:
# Dendrogram
fig_dendrogram = ff.create_dendrogram(linkage_matrix_single)
fig_dendrogram.update_layout(title='Dendrogram (Single Linkage)',
                             xaxis_title='Sample index',
                             yaxis_title='Distance',
                             width=800, height=500)
fig_dendrogram.show()

# Plot Centroid
fig_centroid = px.bar(centroids_single, x='JUMLAH_LONGSOR', y='cluster_single', orientation='h',
                      labels={'JUMLAH_LONGSOR': 'Nilai Centroid', 'cluster_single': 'Cluster'},
                      title='Centroid untuk Setiap Cluster (Metode Single)',
                      color='cluster_single')
fig_centroid.show()


In [26]:
threshold_low_single = 2
threshold_high_single = 32

def get_landslide_category(cluster):
    if cluster in centroids_single['cluster_single'].values:
        mean_value = centroids_single.loc[centroids_single['cluster_single'] == cluster, 'JUMLAH_LONGSOR'].values[0]
        if mean_value < threshold_low_single:
            return 'Tingkat Rawan Rendah'
        elif mean_value < threshold_high_single:
            return 'Tingkat Rawan Sedang'
        else:
            return 'Tingkat Rawan Tinggi'
    else:
        return 'Unknown Cluster'

X_ahc_single['Landslide Category Single'] = X_ahc_single['cluster_single'].map(get_landslide_category)
print(X_ahc_single.head())


   JUMLAH_LONGSOR  JIWA_TERDAMPAK  JIWA_MENINGGAL  RUSAK_TERDAMPAK  \
0             225            2673               0              405   
1             147             311               3               82   
2              27             227               1               48   
3              60             285               3               44   
4              42             294               0               17   

   RUSAK_RINGAN  RUSAK_SEDANG  RUSAK_BERAT  TERTIMBUN  cluster_single  \
0           213           143           49          0               0   
1            24            39           19         14               1   
2            14            24           10          0               1   
3            17            13           14         15               1   
4             1             6           10          2               1   

  Landslide Category Single  
0      Tingkat Rawan Tinggi  
1      Tingkat Rawan Sedang  
2      Tingkat Rawan Sedang  
3      Tingkat Rawan

In [27]:
# Menyimpan CCC ke file CSV
X_ahc_single_csv_path = 'X_ahc_single.csv'
X_ahc_single.to_csv(X_ahc_single_csv_path, index=False)
print(f"AHC SINGLE telah disimpan ke: {X_ahc_single_csv_path}")

AHC SINGLE telah disimpan ke: X_ahc_single.csv


# LINKAGE AVERAGE

In [28]:
linkage_matrix_average = linkage(numeric_data, method='average')
cophenet_matrix_average, _ = cophenet(linkage_matrix_average, pdist(numeric_data))
ccc_average = cophenet_matrix_average.mean()
print("Cophenetic Correlation Coefficient (CCC) untuk metode 'average':", ccc_average)


Cophenetic Correlation Coefficient (CCC) untuk metode 'average': 0.9574486823459332


In [29]:
silhouette_scores = []
n_clusters_range = range(2, 11)
for n_clusters in n_clusters_range:
    clusterer_average = AgglomerativeClustering(n_clusters=n_clusters, linkage='average')
    cluster_labels = clusterer_average.fit_predict(numeric_data)
    silhouette_avg = silhouette_score(numeric_data, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"Silhouette Score untuk metode 'average' dengan {n_clusters} clusters: {silhouette_avg}")

# Menentukan jumlah cluster optimal
optimal_clusters = n_clusters_range[silhouette_scores.index(max(silhouette_scores))]
print(f"Jumlah cluster optimal berdasarkan Silhouette Score adalah: {optimal_clusters}")


Silhouette Score untuk metode 'average' dengan 2 clusters: 0.8831295055993396
Silhouette Score untuk metode 'average' dengan 3 clusters: 0.8133013977269692
Silhouette Score untuk metode 'average' dengan 4 clusters: 0.809411415306873
Silhouette Score untuk metode 'average' dengan 5 clusters: 0.5841276900326505
Silhouette Score untuk metode 'average' dengan 6 clusters: 0.5774563711123003
Silhouette Score untuk metode 'average' dengan 7 clusters: 0.6437658467951861
Silhouette Score untuk metode 'average' dengan 8 clusters: 0.6058932800781461
Silhouette Score untuk metode 'average' dengan 9 clusters: 0.610574056170496
Silhouette Score untuk metode 'average' dengan 10 clusters: 0.5874929667176965
Jumlah cluster optimal berdasarkan Silhouette Score adalah: 2


In [30]:
clusterer_average_optimal = AgglomerativeClustering(n_clusters=optimal_clusters, linkage='average')
X_ahc_average = X_ahc.copy()
X_ahc_average['cluster_average'] = clusterer_average_optimal.fit_predict(numeric_data)


In [31]:
# Linkage Matrix
linkage_df_average = pd.DataFrame(linkage_matrix_average, columns=['Cluster1', 'Cluster2', 'Distance', 'SampleCount'])
linkage_df_average['Cluster1'] = linkage_df_average['Cluster1'].astype(int)
linkage_df_average['Cluster2'] = linkage_df_average['Cluster2'].astype(int)
linkage_csv_path = 'linkage_matrix_average.csv'
linkage_df_average.to_csv(linkage_csv_path, index=False)
print(f"Linkage Matrix (Average Linkage) telah disimpan ke: {linkage_csv_path}")

# Centroid
centroids_average = X_ahc_average.groupby('cluster_average')[['JUMLAH_LONGSOR']].mean().reset_index()
centroids_csv_path = 'centroids_average.csv'
centroids_average.to_csv(centroids_csv_path, index=True)
print(f"Centroid untuk setiap cluster telah disimpan ke: {centroids_csv_path}")


Linkage Matrix (Average Linkage) telah disimpan ke: linkage_matrix_average.csv
Centroid untuk setiap cluster telah disimpan ke: centroids_average.csv


In [32]:
# Dendrogram
fig_dendrogram = ff.create_dendrogram(linkage_matrix_average)
fig_dendrogram.update_layout(title='Dendrogram (Average Linkage)',
                             xaxis_title='Sample index',
                             yaxis_title='Distance',
                             width=800, height=500)
fig_dendrogram.show()

# Plot Centroid
fig_centroid = px.bar(centroids_average, x='JUMLAH_LONGSOR', y='cluster_average', orientation='h',
                      labels={'JUMLAH_LONGSOR': 'Nilai Centroid', 'cluster_average': 'Cluster'},
                      title='Centroid untuk Setiap Cluster (Metode Average)',
                      color='cluster_average')
fig_centroid.show()


In [33]:
threshold_low_average = 2
threshold_high_average = 32

def get_landslide_category(cluster):
    if cluster in centroids_average['cluster_average'].values:
        mean_value = centroids_average.loc[centroids_average['cluster_average'] == cluster, 'JUMLAH_LONGSOR'].values[0]
        if mean_value < threshold_low_average:
            return 'Tingkat Rawan Rendah'
        elif mean_value < threshold_high_average:
            return 'Tingkat Rawan Sedang'
        else:
            return 'Tingkat Rawan Tinggi'
    else:
        return 'Unknown Cluster'

X_ahc_average['Landslide Category Average'] = X_ahc_average['cluster_average'].map(get_landslide_category)

print(X_ahc_average.head())


   JUMLAH_LONGSOR  JIWA_TERDAMPAK  JIWA_MENINGGAL  RUSAK_TERDAMPAK  \
0             225            2673               0              405   
1             147             311               3               82   
2              27             227               1               48   
3              60             285               3               44   
4              42             294               0               17   

   RUSAK_RINGAN  RUSAK_SEDANG  RUSAK_BERAT  TERTIMBUN  cluster_average  \
0           213           143           49          0                0   
1            24            39           19         14                1   
2            14            24           10          0                1   
3            17            13           14         15                1   
4             1             6           10          2                1   

  Landslide Category Average  
0       Tingkat Rawan Tinggi  
1       Tingkat Rawan Sedang  
2       Tingkat Rawan Sedang  
3       Ti

In [34]:
# Menyimpan CCC ke file CSV
X_ahc_average_csv_path = 'X_ahc_average.csv'
X_ahc_average.to_csv(X_ahc_average_csv_path, index=False)
print(f"AHC AVERAGE telah disimpan ke: {X_ahc_average_csv_path}")

AHC AVERAGE telah disimpan ke: X_ahc_average.csv


# LINKAGE COMPLETE

In [35]:
linkage_matrix_complete = linkage(numeric_data, method='complete')
cophenet_matrix_complete, _ = cophenet(linkage_matrix_complete, pdist(numeric_data))
ccc_complete = cophenet_matrix_complete.mean()
print("Cophenetic Correlation Coefficient (CCC) untuk metode 'complete':", ccc_complete)


Cophenetic Correlation Coefficient (CCC) untuk metode 'complete': 0.9583810315643562


In [36]:
silhouette_scores = []
n_clusters_range = range(2, 11)
for n_clusters in n_clusters_range:
    clusterer_complete = AgglomerativeClustering(n_clusters=n_clusters, linkage='complete')
    cluster_labels = clusterer_complete.fit_predict(numeric_data)
    silhouette_avg = silhouette_score(numeric_data, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"Silhouette Score untuk metode 'complete' dengan {n_clusters} clusters: {silhouette_avg}")

# Menentukan jumlah cluster optimal
optimal_clusters = n_clusters_range[silhouette_scores.index(max(silhouette_scores))]
print(f"Jumlah cluster optimal berdasarkan Silhouette Score adalah: {optimal_clusters}")


Silhouette Score untuk metode 'complete' dengan 2 clusters: 0.8831295055993396
Silhouette Score untuk metode 'complete' dengan 3 clusters: 0.8133013977269692
Silhouette Score untuk metode 'complete' dengan 4 clusters: 0.809411415306873
Silhouette Score untuk metode 'complete' dengan 5 clusters: 0.6090813387943331
Silhouette Score untuk metode 'complete' dengan 6 clusters: 0.6520706716491286
Silhouette Score untuk metode 'complete' dengan 7 clusters: 0.6453993527287782
Silhouette Score untuk metode 'complete' dengan 8 clusters: 0.6325645825128456
Silhouette Score untuk metode 'complete' dengan 9 clusters: 0.6264357297929725
Silhouette Score untuk metode 'complete' dengan 10 clusters: 0.5720636002859462
Jumlah cluster optimal berdasarkan Silhouette Score adalah: 2


In [37]:
clusterer_complete_optimal = AgglomerativeClustering(n_clusters=optimal_clusters, linkage='complete')
X_ahc_complete = X_ahc.copy()
X_ahc_complete['cluster_complete'] = clusterer_complete_optimal.fit_predict(numeric_data)


In [38]:
# Linkage Matrix
linkage_df_complete = pd.DataFrame(linkage_matrix_complete, columns=['Cluster1', 'Cluster2', 'Distance', 'SampleCount'])
linkage_df_complete['Cluster1'] = linkage_df_complete['Cluster1'].astype(int)
linkage_df_complete['Cluster2'] = linkage_df_complete['Cluster2'].astype(int)
linkage_csv_path = 'linkage_matrix_complete.csv'
linkage_df_complete.to_csv(linkage_csv_path, index=False)
print(f"Linkage Matrix (Complete Linkage) telah disimpan ke: {linkage_csv_path}")

# Centroid
centroids_complete = X_ahc_complete.groupby('cluster_complete')[['JUMLAH_LONGSOR']].mean().reset_index()
centroids_csv_path = 'centroids_complete.csv'
centroids_complete.to_csv(centroids_csv_path, index=True)
print(f"Centroid untuk setiap cluster telah disimpan ke: {centroids_csv_path}")


Linkage Matrix (Complete Linkage) telah disimpan ke: linkage_matrix_complete.csv
Centroid untuk setiap cluster telah disimpan ke: centroids_complete.csv


In [39]:
# Dendrogram
fig_dendrogram = ff.create_dendrogram(linkage_matrix_complete)
fig_dendrogram.update_layout(title='Dendrogram (Complete Linkage)',
                             xaxis_title='Sample index',
                             yaxis_title='Distance',
                             width=800, height=500)
fig_dendrogram.show()

# Plot Centroid
fig_centroid = px.bar(centroids_complete, x='JUMLAH_LONGSOR', y='cluster_complete', orientation='h',
                      labels={'JUMLAH_LONGSOR': 'Nilai Centroid', 'cluster_complete': 'Cluster'},
                      title='Centroid untuk Setiap Cluster (Metode Complete)',
                      color='cluster_complete')
fig_centroid.show()


In [40]:
threshold_low_complete = 2
threshold_high_complete = 32

def get_landslide_category(cluster):
    if cluster in centroids_complete['cluster_complete'].values:
        mean_value = centroids_complete.loc[centroids_complete['cluster_complete'] == cluster, 'JUMLAH_LONGSOR'].values[0]
        if mean_value < threshold_low_complete:
            return 'Tingkat Rawan Rendah'
        elif mean_value < threshold_high_complete:
            return 'Tingkat Rawan Sedang'
        else:
            return 'Tingkat Rawan Tinggi'
    else:
        return 'Unknown Cluster'

X_ahc_complete['Landslide Category Complete'] = X_ahc_complete['cluster_complete'].map(get_landslide_category)

print(X_ahc_complete.head())


   JUMLAH_LONGSOR  JIWA_TERDAMPAK  JIWA_MENINGGAL  RUSAK_TERDAMPAK  \
0             225            2673               0              405   
1             147             311               3               82   
2              27             227               1               48   
3              60             285               3               44   
4              42             294               0               17   

   RUSAK_RINGAN  RUSAK_SEDANG  RUSAK_BERAT  TERTIMBUN  cluster_complete  \
0           213           143           49          0                 0   
1            24            39           19         14                 1   
2            14            24           10          0                 1   
3            17            13           14         15                 1   
4             1             6           10          2                 1   

  Landslide Category Complete  
0        Tingkat Rawan Tinggi  
1        Tingkat Rawan Sedang  
2        Tingkat Rawan Sedang  


In [41]:
# Menyimpan CCC ke file CSV
X_ahc_complete_csv_path = 'X_ahc_complete.csv'
X_ahc_complete.to_csv(X_ahc_complete_csv_path, index=False)
print(f"AHC COMPLETE telah disimpan ke: {X_ahc_complete_csv_path}")

AHC COMPLETE telah disimpan ke: X_ahc_complete.csv


# PERBANDINGAN ANTARA KETIGA METODE DENGAN MENGGUNAKAN CCC DAN SC

In [46]:
# Menghitung CCC untuk metode 'single'
linkage_matrix_single = linkage(numeric_data, method='single')
cophenet_matrix_single, _ = cophenet(linkage_matrix_single, pdist(numeric_data))
ccc_single = cophenet_matrix_single.mean()

# Menghitung CCC untuk metode 'average'
linkage_matrix_average = linkage(numeric_data, method='average')
cophenet_matrix_average, _ = cophenet(linkage_matrix_average, pdist(numeric_data))
ccc_average = cophenet_matrix_average.mean()

# Menghitung CCC untuk metode 'complete'
linkage_matrix_complete = linkage(numeric_data, method='complete')
cophenet_matrix_complete, _ = cophenet(linkage_matrix_complete, pdist(numeric_data))
ccc_complete = cophenet_matrix_complete.mean()

# Membuat DataFrame untuk perbandingan CCC
ccc_comparison = {
    'Metode': ['Single', 'Average', 'Complete'],
    'CCC': [ccc_single, ccc_average, ccc_complete]
}
ccc_comparison_df = pd.DataFrame(ccc_comparison)

# Menyimpan CCC ke file CSV
ccc_comparison_csv_path = 'ccc_comparison.csv'
ccc_comparison_df.to_csv(ccc_comparison_csv_path, index=False)
print(f"Perbandingan CCC telah disimpan ke: {ccc_comparison_csv_path}")




Perbandingan CCC telah disimpan ke: ccc_comparison.csv


In [47]:
print("Perbandingan Cophenetic Correlation Coefficient (CCC):")
print(ccc_comparison_df)

# Membuat plot perbandingan CCC
fig_ccc = px.bar(ccc_comparison_df, x='Metode', y='CCC', 
                 title='Perbandingan Cophenetic Correlation Coefficient (CCC)',
                 labels={'CCC': 'Cophenetic Correlation Coefficient', 'Metode': 'Metode Clustering'},
                 color='Metode',
                 color_discrete_map={
                     'Single': 'blue',
                     'Average': 'green',
                     'Complete': 'red'
                 })

# Menampilkan plot
fig_ccc.show()

Perbandingan Cophenetic Correlation Coefficient (CCC):
     Metode       CCC
0    Single  0.947359
1   Average  0.957449
2  Complete  0.958381


In [43]:
silhouette_scores_single = []
silhouette_scores_average = []
silhouette_scores_complete = []
n_clusters_range = range(2, 11)

for n_clusters in n_clusters_range:
    # Metode 'single'
    clusterer_single = AgglomerativeClustering(n_clusters=n_clusters, linkage='single')
    cluster_labels_single = clusterer_single.fit_predict(numeric_data)
    silhouette_avg_single = silhouette_score(numeric_data, cluster_labels_single)
    silhouette_scores_single.append(silhouette_avg_single)
    
    # Metode 'average'
    clusterer_average = AgglomerativeClustering(n_clusters=n_clusters, linkage='average')
    cluster_labels_average = clusterer_average.fit_predict(numeric_data)
    silhouette_avg_average = silhouette_score(numeric_data, cluster_labels_average)
    silhouette_scores_average.append(silhouette_avg_average)
    
    # Metode 'complete'
    clusterer_complete = AgglomerativeClustering(n_clusters=n_clusters, linkage='complete')
    cluster_labels_complete = clusterer_complete.fit_predict(numeric_data)
    silhouette_avg_complete = silhouette_score(numeric_data, cluster_labels_complete)
    silhouette_scores_complete.append(silhouette_avg_complete)

# Menyimpan hasil Silhouette Score ke dalam DataFrame
silhouette_df = pd.DataFrame({
    'Jumlah Cluster': n_clusters_range,
    'Single Linkage': silhouette_scores_single,
    'Average Linkage': silhouette_scores_average,
    'Complete Linkage': silhouette_scores_complete
})

# Menyimpan Silhouette Score ke file CSV
silhouette_scores_csv_path = 'silhouette_scores.csv'
silhouette_df.to_csv(silhouette_scores_csv_path, index=False)
print(f"Silhouette Score telah disimpan ke: {silhouette_scores_csv_path}")


Silhouette Score telah disimpan ke: silhouette_scores.csv


In [48]:
silhouette_df

,Jumlah Cluster,Single Linkage,Average Linkage,Complete Linkage
0,2,0.883130,0.883130,0.883130
1,3,0.813301,0.813301,0.813301
2,4,0.809411,0.809411,0.809411
3,5,0.802740,0.584128,0.609081
4,6,0.255140,0.577456,0.652071
5,7,0.288677,0.643766,0.645399
6,8,0.268253,0.605893,0.632565
7,9,0.271844,0.610574,0.626436
8,10,0.221134,0.587493,0.572064


In [50]:
# Menampilkan plot Silhouette Score terhadap jumlah cluster untuk masing-masing metode
fig = px.line(silhouette_df, x='Jumlah Cluster', y=['Single Linkage', 'Average Linkage', 'Complete Linkage'],
              labels={'value': 'Silhouette Score', 'variable': 'Metode'},
              title='Silhouette Score untuk Berbagai Jumlah Cluster',
              color_discrete_map={
                  'Single Linkage': 'blue',
                  'Average Linkage': 'green',
                  'Complete Linkage': 'red'
              })

# Tampilkan plot
fig.show()